# Create and run Earthquake Estimator

In [2]:
!nvidia-smi

Mon Jan 21 06:39:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
from importlib import reload
import time
import os

In [4]:
EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords2'

In [5]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        del lstm_estimator
        del earthquake_input_fn
    import lstm_estimator
    import earthquake_input_fn

In [13]:
reload_tf()

In [7]:
def create_estimator_from_params(batch_size, timesteps, n_feats, feature_columns, lstm_cell_size, learning_rate):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'n_feats': n_feats,
        'feature_columns': feature_columns,
        'lstm_cell_size': lstm_cell_size,
        'learning_rate': learning_rate
    }
    model_dir = '/workspace/persistent-data/models/%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    estim = tf.estimator.Estimator(model_fn=lstm_estimator.lstm_model_fn,
                                   params=params,
                                   model_dir=model_dir)
    return estim

In [8]:
BATCH_SIZE=64
TIMESTEPS=16
LSTM_CELL_SIZE=128
LEARNING_RATE=0.0001

In [9]:
estimator = create_estimator_from_params(batch_size=BATCH_SIZE,
                                         timesteps=TIMESTEPS,
                                         n_feats=1,
                                         feature_columns=[tf.feature_column.numeric_column(key='acousticdata')],
                                         lstm_cell_size=LSTM_CELL_SIZE,
                                         learning_rate=LEARNING_RATE)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_model_dir': '/workspace/persistent-data/models/2019-01-21-06-39-45', '_task_id': 0, '_evaluation_master': '', '_num_ps_replicas': 0, '_task_type': 'worker', '_tf_random_seed': None, '_device_fn': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_summary_steps': 100, '_train_distribute': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f547e6a0780>, '_experimental_distribute': None, '_eval_distribute': None, '_service': None, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_protocol': None, '_num_worker_replicas': 1, '_is_chief': True}


In [ ]:
DISTINCT_BATCHES = 1300
STEPS_PER_BATCH = int(4096/TIMESTEPS)
TOTAL_STEPS = DISTINCT_BATCHES * STEPS_PER_BATCH
print('running estimator for %d steps' % TOTAL_STEPS)
estimator.train(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(EARTHQUAKE_DATA_DIR,
                                                                          BATCH_SIZE,
                                                                          TIMESTEPS,
                                                                          scales=['1e1', '1e0'],
                                                                          traintest='train'),
                steps=TOTAL_STEPS)

running estimator for 332800 steps
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /workspace/persistent-data/models/2019-01-21-06-39-45/model.ckpt-66560
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 66560 into /workspace/persistent-data/models/2019-01-21-06-39-45/model.ckpt.
INFO:tensorflow:loss = 4049.830737303083, step = 66560
INFO:tensorflow:global_step/sec: 20.6653
INFO:tensorflow:loss = 50.895711365888616, step = 66660 (4.842 sec)
INFO:tensorflow:global_step/sec: 27.999
INFO:tensorflow:loss = 43.657740513308624, step = 66760 (3.573 sec)
INFO:tensorflow:global_step/sec: 29.0175
INFO:tensorflow:loss = 62.18700086935793, step = 66860 (3.445 sec)
INFO:tensorflow:global_step/sec: 28.3448
INFO:tensorflow:loss = 32.19221684188815, step = 66960 (3.528 sec)
INFO:tensor